In [2]:
data = sc.textFile('../../data/msnbc/data50k.csv') \
         .map(lambda line: list(map(int,line.split(',')))) \
         .zipWithIndex() \
         .map(lambda x: (x[1], x[0]))

In [3]:
dataset = data.sortByKey().collect()

In [4]:
dataset = sc.broadcast(dataset)

In [5]:
dataset.value[1]

(1, [2])

In [6]:
# lines = open("../../data/msnbc/data.csv", 'r').readlines()[:80000]
# open("../../data/msnbc/data50k.csv", 'w').writelines(lines)

## getMinSupItems

### itemlocmap

In [7]:
def get_indices_for(x):
    key = x[0]
    seq_id = x[1]
    id, seq = dataset.value[seq_id]
    indices = list()
    for idx in range(len(seq)):
        if seq[idx] == key:
            indices.append(idx)
    return key, seq_id, indices

In [8]:
def get_first_last_index(x):
    key = x[0]
    seq_id = x[1]
    indices = x[2]
    return key, (seq_id, indices[0], indices[-1])

In [9]:
def get_freq(x):
    key = x[0]
    sequences = x[1]
    freq = len(sequences)
    return key, freq, sequences

In [10]:
itemlocmap = data.map(lambda x: (x[0], set(x[1])))  \
        .flatMapValues(lambda x:x) \
        .map(lambda x: (x[1], x[0])) \
        .map(lambda x: get_indices_for(x)) \
        .map(lambda x: get_first_last_index(x)) \
        .groupByKey() \
        .mapValues(list) \
        .map(lambda x: get_freq(x))

In [11]:
result = itemlocmap.collect()

In [12]:
# result[0]

### above_threshold

In [13]:
THRESHOLD = 50

In [14]:
def above_threshold_filter(x, threshold=THRESHOLD):
    return x[1] > threshold

In [15]:
above_threshold = itemlocmap.filter(lambda x: above_threshold_filter(x, THRESHOLD)) \
                            .map(lambda x: (x[0], (x[1:])))

In [16]:
items = sc.broadcast(above_threshold.sortByKey().collect())

In [17]:
# items.value[5]

In [18]:
# result = above_threshold.collect()

In [19]:
def extract_seq(x):
    item, (freq, seq) = x
    return item, [y[0] for y in seq]

In [20]:
item_sequences = sc.broadcast(above_threshold.map(lambda x: extract_seq(x)).sortByKey().collect())

In [21]:
# item_sequences.value[0]

In [22]:
items_ids_above_threshold = sc.broadcast(above_threshold.map(lambda x:x[0]).sortBy(lambda x:x).collect())

In [23]:
items_ids_above_threshold.value

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

## genRules

In [24]:
items_ids = above_threshold.map(lambda x: x[0]) \
                           .zipWithIndex()
                           

In [25]:
count_items = items_ids.count()

In [26]:
result = items_ids.collect()

In [27]:
result

[(2, 0),
 (4, 1),
 (6, 2),
 (8, 3),
 (10, 4),
 (12, 5),
 (14, 6),
 (16, 7),
 (1, 8),
 (3, 9),
 (5, 10),
 (7, 11),
 (9, 12),
 (11, 13),
 (13, 14),
 (15, 15),
 (17, 16)]

### Join

In [28]:
def prepare_join(x, count):
    return x, list(range(x[1] + 1, count))

In [29]:
items_join = items_ids.map(lambda x: prepare_join(x, count_items)) \
                      .flatMapValues(lambda x:x) \
                      .map(lambda x: (x[0][1], x[1])) 


In [30]:
result = items_join.collect()
result[5:12]

[(0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12)]

### I => J combination with no repeats

In [31]:
def prepare_common_sequences(x):
    i = x[0]
    j = x[1]
    itemI, itemJ = items.value[i], items.value[j]
    occurancesI, occurancesJ = itemI[1][1], itemJ[1][1]
    
    allseqboth = []
    
    for seq_idI, firstI, lastI in occurancesI:
        for seq_idJ, firstJ, lastJ in occurancesJ:
            if seq_idI == seq_idJ:
                allseqboth.append((seq_idI, (firstI, lastI), (firstJ, lastJ)))
    
    return x, allseqboth

In [32]:
def count_IJ_JI_rules(x): 
    key = x[0]
    seq_id, itemI, itemJ = x[1]
    
    IJ = []
    JI = []
    if itemI[0] < itemJ[1]:
        IJ.append(seq_id)
    if itemJ[0] < itemI[1]:
        JI.append(seq_id)
        
    return key, [IJ, JI]

In [33]:
def group_IJ_JI_rules(a, b):
    a[0] += b[0]
    a[1] += b[1]
    return a

In [34]:
def add_index_info(x):
    key = x[0]
    valuesI, valuesJ = x[1]
    return key, ((valuesI, key), (valuesJ, tuple(reversed(key))))

In [35]:
reduced_IJ_JI = items_join.map(lambda x: prepare_common_sequences(x)) \
                    .flatMapValues(lambda x:x) \
                    .map(lambda x: count_IJ_JI_rules(x)) \
                    .reduceByKey(group_IJ_JI_rules) \
                    .map(lambda x: add_index_info(x)) 


In [37]:
# result = reduced_IJ_JI.collect()
# result[:2]

### Accumulate rules

In [38]:
MIN_SUP_REL = 0.01 * len(dataset.value)
MIN_CONF = 0.1

In [39]:
def get_freq(x):
    key = x[0]
    values = x[1]
    frequencies = len(values)
    return key, (values, frequencies)

def is_above_minsup_relative(x, minsup_relative):
    return x[1][1] > minsup_relative 

In [40]:
def generate_rule_and_expands(x, min_conf=MIN_CONF):
    i, j = x[0]
    allseqIJ, length = x[1]
    itemI, itemJ = items.value[i], items.value[j]
    occurancesI, occurancesJ = itemI[1][1], itemJ[1][1]
#     occurancesI = (seq_id, first, last)
    
    allseqI = [x[0] for x in occurancesI]
    allseqJ = [x[0] for x in occurancesJ]
    
    confIJ = length / len(occurancesI)
    
    antecedentSet = set([i])
    consequentSet = set([j])
    
    rules = []
    if confIJ >= min_conf:
        rules.append((antecedentSet, consequentSet, length / len(dataset.value), confIJ))
        
#     expandLeft(antecedentSet,consequentSet,allseqI,allseqIJ,occurancesJ)
#     expandRight(antecedentSet,consequentSet,allseqI,allseqJ,allseqIJ,occurancesI,occurancesJ)

    expand_left = (0, antecedentSet, consequentSet, allseqI, occurancesJ, allseqIJ)
    expand_right = (1, antecedentSet, consequentSet, allseqI, allseqJ, occurancesI, occurancesJ, allseqIJ)
    return rules, expand_left, expand_right

In [41]:
rules_and_expands = reduced_IJ_JI.flatMapValues(lambda x:x) \
                     .map(lambda x: (x[1][1], x[1][0])) \
                     .map(lambda x: get_freq(x)) \
                     .filter(lambda x: is_above_minsup_relative(x, MIN_SUP_REL)) \
                     .map(lambda x: generate_rule_and_expands(x, MIN_CONF))


In [45]:
# result = rules_and_expands.collect()
# result[:1]

In [46]:
# rules = rules_and_expands.map(lambda x: x[0]) \
#                          .filter(lambda x: len(x) > 0) \
#                          .collect()

In [47]:
# expands = rules_and_expands.map(lambda x: (x[1], x[2])) \
#                            .flatMap(lambda x: x)

In [48]:
# result = expands.collect()

## Expand Left - Right

In [49]:
def expand(x, min_sup_rel=MIN_SUP_REL, min_conf=MIN_CONF):
    if x[0] == 0:
        return expand_left(x[1:], min_sup_rel, min_conf)
    else:
        return expand_right(x[1:], min_sup_rel, min_conf)

    
def item_suitable_for_antecedentSet(item, index_item, antecedentSet, consequentSet):
    for i in antecedentSet:
        if i >= index_item:
            return False        
    if index_item in consequentSet or item not in items_ids_above_threshold.value:
        return False
    return True

In [50]:
def index_of_item(item):
    for i, j in enumerate(items_ids_above_threshold.value):
        if j == item:
            return i
    return len(items_ids_above_threshold.value)

In [51]:
def find_first_last(occurances, seq_id):
    for id, first, last in occurances:
        if id == seq_id:
            return first, last

In [52]:
def expand_left(x, min_sup_rel, min_conf):
    antecedentSet, consequentSet, allseqI, occurancesJ, allseqIJ = x
    
    possibleC = dict()
    rules = []
    expand_lefts = []
    seqsLeft = len(allseqIJ)
    
    for seqID in allseqIJ:
        _, seq = dataset.value[seqID] # Get sequence
        firstJ, lastJ = find_first_last(occurancesJ, seqID) # Get last occurance of J in sequene
        
        for item in seq[:lastJ]:
            index_item = index_of_item(item)
            if not item_suitable_for_antecedentSet(item, index_item, antecedentSet, consequentSet):
                continue
                
            if index_item not in possibleC: # first time item was found
                if seqsLeft >= min_sup_rel: # min_sup_rel can be accomplished
                    possibleC[index_item] = set([seqID])
            elif len(possibleC[index_item]) + seqsLeft < min_sup_rel: # there no enough seqLeft to accomplish min_sup
                del possibleC[index_item]
            else:
                possibleC[index_item].add(seqID)
                
        seqsLeft -= 1
         
    # Loop through possibleC to generate valid rules
    for itemC, seqIDs in possibleC.items():
        # Check if minimum support requirement is met
        if len(seqIDs) >= min_sup_rel:
            # SeqIDs of IuC 
            item, seqC = item_sequences.value[itemC]
            allseqIC = set.intersection(set(seqC),allseqI)
            
            confIC_J = len(seqIDs) / len(allseqIC)

            itemsIC = antecedentSet.copy()
            itemsIC.add(itemC)

            if confIC_J >= min_conf:
                rules.append((itemsIC,consequentSet,len(seqIDs)/len(dataset.value),confIC_J))
                
            expand_lefts.append((0, itemsIC,consequentSet, allseqIC, occurancesJ, seqIDs))
    return rules, expand_lefts
            

In [53]:
def expand_right(x, min_sup_rel, min_conf):
    antecedentSet, consequentSet, allseqI, allseqJ, occurancesI, occurancesJ, allseqIJ = x

    possibleC = dict()
    rules = []
    expand_lefts = []
    expand_rights = []
    seqsLeft = len(allseqIJ)
    
    for seqID in allseqIJ:
        _, seq = dataset.value[seqID] # Get sequence
        firstI, lastI = find_first_last(occurancesI, seqID) # Get last occurance of J in sequene
    
        for item in seq[firstI+1:]:
            index_item = index_of_item(item)
            if not item_suitable_for_antecedentSet(item, index_item, consequentSet, antecedentSet):
                continue
            if index_item not in possibleC: # first time item was found
                if seqsLeft >= min_sup_rel: # min_sup_rel can be accomplished
                    possibleC[index_item] = set([seqID])
            elif len(possibleC[index_item]) + seqsLeft < min_sup_rel: # there no enough seqLeft to accomplish min_sup
                del possibleC[index_item]
            else:
                possibleC[index_item].add(seqID)
                
        seqsLeft -= 1
    
    for itemC, seqIDs in possibleC.items():
        if len(seqIDs) >= min_sup_rel:
            
            allseqJC = set()
            # New consequent occurance map
            occurancesJC = dict()
            
            for seqID_J in allseqJ:
                item, seqC = item_sequences.value[itemC]
                if seqID_J in seqC:
                    firstC, lastC = find_first_last(items.value[itemC][1][1], seqID_J)
                    allseqJC.add(seqID_J)
                    firstJ, lastJ = find_first_last(occurancesJ,seqID_J)
                    if lastC < lastJ:
                        occurancesJC[seqID_J] = [seqID_J,firstC,lastC]
                    else:
                        occurancesJC[seqID_J] = [seqID_J,firstJ,lastJ]
                        
            occurancesJC = list(occurancesJC.values())
            confI_JC = len(seqIDs) / len(allseqI)  
            itemsJC = consequentSet.copy()
            itemsJC.add(itemC)
            
            if confI_JC >= min_conf:
                rules.append((antecedentSet,itemsJC,len(seqIDs)/len(dataset.value),confI_JC))
                
            expand_lefts.append((0, antecedentSet, itemsJC, allseqI,occurancesJC, seqIDs))
            expand_rights.append((1, antecedentSet, itemsJC, allseqI, allseqJC, occurancesI, occurancesJC, seqIDs))
                
    return rules,  expand_rights + expand_lefts



## Compute Rules

In [54]:
MIN_SUP_REL = 0.005 * len(dataset.value)
MIN_CONF = 0.001

In [55]:
rules_and_expands = reduced_IJ_JI.flatMapValues(lambda x:x) \
                     .map(lambda x: (x[1][1], x[1][0])) \
                     .map(lambda x: get_freq(x)) \
                     .filter(lambda x: is_above_minsup_relative(x, MIN_SUP_REL)) \
                     .map(lambda x: generate_rule_and_expands(x, MIN_CONF))


In [ ]:
rules = rules_and_expands.map(lambda x: x[0]) \
                         .filter(lambda x: len(x) > 0) \
                         .collect()

expands = rules_and_expands.map(lambda x: (x[1], x[2])) \
                           .flatMap(lambda x: x)

In [ ]:
print(f'Found {len(rules)} rules')
# print(len(expands.collect()))

In [ ]:
while True:
    rules_and_expands = expands.map(lambda x: expand(x, MIN_SUP_REL, MIN_CONF))
    rules += rules_and_expands.map(lambda x: x[0]) \
                          .filter(lambda x: len(x) > 0) \
                          .collect()
    expands = rules_and_expands.map(lambda x: x[1]) \
                           .filter(lambda x: len(x) > 0) \
                           .flatMap(lambda x: x) 
    print(f'Found {len(rules)} rules')
#     print(len(expands.collect()))
    if expands.isEmpty():
        break

In [ ]:
all_rules = []
for rule in rules:
    for r in rule:
        all_rules.append(r)
        

In [ ]:
all_rules